<a href="https://colab.research.google.com/github/ngochuyen2609/tv-program-recommender/blob/1/cf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Chuẩn hoá & JOIN**



In [1]:
# CÀI ĐẶT & THIẾT LẬP
!pip install gdown -q
!pip install pyarrow -q
!pip install "dask[dataframe]" -q  # Cài đặt Dask

import pandas as pd
import numpy as np
import os
import dask.dataframe as dd # Import Dask DataFrame
from dask.diagnostics import ProgressBar

# Tạo các thư mục để lưu kết quả
os.makedirs('data/interim', exist_ok=True)

In [2]:
# TẢI DỮ LIỆU

# Dictionary chứa ID và tên file
files_to_download = {
    "dataset11-30.csv": "1oxX2lMwmMHHXJHQFAs94I-Wxl57ckX7L",
    "export_arh_11-20-final.csv": "1cSNZhAevGeL4yVDOG8Rpao4qhQFEaqAL",
    "export_arh_21-30-final.csv": "1lPLm--x1lhUYbTE1HxZPLMp7dCT6l0n5",
    "export_arh_31-42-final.csv": "1SGSEgSbm4Lniy9AJWHzNH3HEQ5mA6_Ab",
    "submission.csv": "1nmlPqHj8iYxVIGvVax_SEnwrAkJf5oOj"
}

for filename, file_id in files_to_download.items():
    if "YOUR_FILE_ID" in file_id:
        print(f"   -> CẢNH BÁO: Vui lòng cung cấp ID cho file '{filename}'")
    else:
        print(f"   -> Đang tải {filename}...")
        !gdown --id {file_id} -O {filename}

print("   -> Tải dữ liệu hoàn tất!")

   -> Đang tải dataset11-30.csv...
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1oxX2lMwmMHHXJHQFAs94I-Wxl57ckX7L
From (redirected): https://drive.google.com/uc?id=1oxX2lMwmMHHXJHQFAs94I-Wxl57ckX7L&confirm=t&uuid=9e2cb50d-86b0-4ac4-a161-b7084ecb6621
To: /content/dataset11-30.csv
100% 396M/396M [00:04<00:00, 84.6MB/s]
   -> Đang tải export_arh_11-20-final.csv...
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1cSNZhAevGeL4yVDOG8Rpao4qhQFEaqAL
From (redirected): https://drive.google.com/uc?id=1cSNZhAevGeL4yVDOG8Rpao4qhQFE

In [3]:
# TẢI VÀ CHUẨN HÓA DỮ LIỆU ĐẦU VÀO
import pandas as pd
import numpy as np


# Xử lý file logs (dataset11-30.csv)
try:
    print("   -> Đang xử lý file logs (dataset11-30.csv)...")
    logs_df = pd.read_csv('dataset11-30.csv')
    logs_df = logs_df.rename(columns={'stop_time': 'end_time', 'duraton': 'duration'})

    # Chuyển đổi cột ngày giờ, tự động nhận diện nhiều định dạng
    # (xử lý cả 'DD.MM.YYYY' và 'YYYY-MM-DD')
    logs_df['start_time'] = pd.to_datetime(logs_df['start_time'], format='mixed', dayfirst=True)
    logs_df['end_time'] = pd.to_datetime(logs_df['end_time'], format='mixed', dayfirst=True)
    print("      - Chuyển đổi cột ngày giờ trong logs thành công.")

    # Lọc các lượt xem có thời lượng >= 180s
    MIN_DURATION = 180
    logs_filtered_df = logs_df[logs_df['duration'] >= MIN_DURATION].copy()

    # Đổi tên cột để chuẩn bị cho việc join
    logs_filtered_df = logs_filtered_df.rename(columns={
        'start_time': 'start_time_view',
        'end_time': 'end_time_view',
        'duration': 'duration_view'
    })
    print(f"      - Đã lọc và giữ lại {len(logs_filtered_df)} lượt xem có thời lượng >= {MIN_DURATION}s.")

except FileNotFoundError:
    print("   -> LỖI: Không tìm thấy file 'dataset11-30.csv'. Hãy chắc chắn bạn đã chạy Cell 2 để tải dữ liệu.")
    assert False, "File logs không tồn tại."


# Xử lý file programs (lịch phát sóng)
try:
    print("\n   -> Đang xử lý các file lịch phát sóng...")
    prog_11_20 = pd.read_csv('export_arh_11-20-final.csv')
    prog_21_30 = pd.read_csv('export_arh_21-30-final.csv')

    # Ghép hai file lịch phát sóng lại
    programs_df = pd.concat([prog_11_20, prog_21_30], ignore_index=True)

    # Loại bỏ các chương trình không liên quan (tv_show_id = 0)
    programs_df = programs_df[programs_df['tv_show_id'] != 0].copy()

    # Chuyển đổi cột ngày giờ, tự động nhận diện nhiều định dạng
    programs_df['start_time'] = pd.to_datetime(programs_df['start_time'], format='mixed', dayfirst=True)

    # Đổi tên cột để chuẩn bị cho việc join
    programs_df = programs_df.rename(columns={
        'start_time': 'start_time_prog',
        'duration': 'duration_prog'
    })
    print(f"      - Đã gộp và xử lý {len(programs_df)} chương trình hợp lệ.")

except FileNotFoundError:
    print("   -> LỖI: Không tìm thấy file 'export_arh_...'.")
    assert False, "File lịch phát sóng không tồn tại."

print("\n   -> Chuẩn hóa dữ liệu hoàn tất. Các DataFrame 'logs_filtered_df' và 'programs_df' đã sẵn sàng.")

   -> Đang xử lý file logs (dataset11-30.csv)...
      - Chuyển đổi cột ngày giờ trong logs thành công.
      - Đã lọc và giữ lại 3927546 lượt xem có thời lượng >= 180s.

   -> Đang xử lý các file lịch phát sóng...
      - Đã gộp và xử lý 237118 chương trình hợp lệ.

   -> Chuẩn hóa dữ liệu hoàn tất. Các DataFrame 'logs_filtered_df' và 'programs_df' đã sẵn sàng.


In [4]:
# RIỂN KHAI LOGIC TỪ join_program.py
import pandas as pd
import numpy as np
import gc

def _pick_overlap(logs_ch: pd.DataFrame, meta_ch: pd.DataFrame) -> pd.DataFrame:
    # Trả về dataframe rỗng nếu không có dữ liệu đầu vào
    required_cols = ["user_id","tv_show_id","vsetv_id","start_time_view","end_time_view","duration_view"]
    if logs_ch.empty or meta_ch.empty:
        return pd.DataFrame(columns=required_cols)

    # Chuẩn bị dữ liệu
    meta_ch = meta_ch.sort_values("start_time").reset_index(drop=True)
    logs_ch = logs_ch.copy()
    logs_ch["view_id"] = np.arange(len(logs_ch))

    cand_L = pd.merge_asof(
        logs_ch.sort_values("start_time_view"),
        meta_ch,
        left_on="start_time_view",
        right_on="start_time",
        direction="backward", # Tìm chương trình gần nhất về trước
        allow_exact_matches=True
    )

    cand_R = pd.merge_asof(
        logs_ch.sort_values("end_time_view"),
        meta_ch,
        left_on="end_time_view",
        right_on="start_time",
        direction="backward",
        allow_exact_matches=True
    )

    # Gộp và lọc các ứng viên
    cand = pd.concat([cand_L, cand_R], ignore_index=True).dropna(subset=["tv_show_id"])
    if cand.empty:
        return pd.DataFrame(columns=required_cols)

    # Tính toán thời gian giao nhau (overlap) > 0
    s = np.maximum(cand["start_time_view"].values, cand["start_time"].values)
    e = np.minimum(cand["end_time_view"].values, cand["prog_end"].values)
    overlap_seconds = (e - s).astype('timedelta64[s]').astype(int)

    cand["overlap_s"] = overlap_seconds
    cand = cand[cand["overlap_s"] > 0]
    if cand.empty:
        return pd.DataFrame(columns=required_cols)

    # Chọn ứng viên có overlap lớn nhất cho mỗi lượt xem
    cand = cand.sort_values(["view_id", "overlap_s"], ascending=[True, False])
    best_match = cand.drop_duplicates(subset=["view_id"], keep="first")

    return best_match[required_cols].reset_index(drop=True)

def interval_join_logs_with_program(logs: pd.DataFrame, meta: pd.DataFrame) -> pd.DataFrame:
    all_matches = []
    # Lặp qua từng kênh để giữ mức sử dụng RAM luôn thấp
    channel_groups = logs.groupby("vsetv_id", sort=False)
    num_channels = len(channel_groups)

    print(f"   -> Bắt đầu xử lý {num_channels} kênh một cách tuần tự...")
    for i, (vid, logs_ch) in enumerate(channel_groups):
        meta_ch = meta[meta["channel_id"] == vid]
        matched = _pick_overlap(logs_ch, meta_ch)
        if not matched.empty:
            all_matches.append(matched)
        if (i + 1) % 50 == 0:
             print(f"      - Đã xử lý {i + 1}/{num_channels} kênh...")

    if not all_matches:
        return pd.DataFrame(columns=["user_id","tv_show_id","vsetv_id","start_time_view","end_time_view","duration_view"])

    print("   -> Ghép kết quả từ tất cả các kênh...")
    return pd.concat(all_matches, ignore_index=True)



try:
    if 'prog_end' not in programs_df.columns:
        programs_df['prog_end'] = programs_df['start_time_prog'] + pd.to_timedelta(programs_df['duration_prog'], unit='s')

    programs_df = programs_df.rename(columns={'start_time_prog': 'start_time'})

    joined_df = interval_join_logs_with_program(logs_filtered_df, programs_df)

    # Tính toán và kiểm tra coverage (dựa trên số lượng log)
    coverage = len(joined_df) / len(logs_filtered_df) if len(logs_filtered_df) > 0 else 0
    print(f"\n      - CHECKPOINT: Coverage = {len(joined_df)} / {len(logs_filtered_df)} = {coverage:.3f} (kỳ vọng > 0.70)")

    # Lưu kết quả
    output_path = 'data/interim/logs_program.parquet'
    joined_df.to_parquet(output_path, index=False)
    print(f"   -> Dữ liệu đã được lưu tại: {output_path}")

    # Dọn dẹp RAM
    del logs_filtered_df, programs_df, joined_df
    gc.collect()
    print("   -> Đã dọn dẹp bộ nhớ RAM.")

except NameError:
    print("\n   -> LỖI: Không tìm thấy 'logs_filtered_df' hoặc 'programs_df'.")

   -> Bắt đầu xử lý 200 kênh một cách tuần tự...
      - Đã xử lý 50/200 kênh...
      - Đã xử lý 100/200 kênh...
      - Đã xử lý 150/200 kênh...
      - Đã xử lý 200/200 kênh...
   -> Ghép kết quả từ tất cả các kênh...

      - CHECKPOINT: Coverage = 2122836 / 3927546 = 0.540 (kỳ vọng > 0.70)
   -> Dữ liệu đã được lưu tại: data/interim/logs_program.parquet
   -> Đã dọn dẹp bộ nhớ RAM.


**Session hoá & Split thời gian**

In [5]:
# SESSIONIZE VÀ SPLIT
input_path = 'data/interim/logs_program.parquet'
try:
    df = pd.read_parquet(input_path)
    df['start_time_view'] = pd.to_datetime(df['start_time_view'])
    print(f"   -> Đã tải thành công {len(df)} bản ghi từ '{input_path}'.")
except FileNotFoundError:
    print(f"   -> LỖI: Không tìm thấy file '{input_path}'.")
    # Dừng thực thi nếu file không tồn tại
    assert False, "File đầu vào không tồn tại."


print("   -> Đang tạo session...")
df = df.sort_values(['user_id', 'start_time_view']).reset_index(drop=True)
df['time_gap'] = df.groupby('user_id')['start_time_view'].diff()
# Sử dụng giá trị mặc định từ script của bạn
GAP_MINUTES = 45
SESSION_GAP = pd.Timedelta(f'{GAP_MINUTES} minutes')
is_new_session = (df['time_gap'] > SESSION_GAP)
df['session_id'] = is_new_session.cumsum()
print(f"      - Đã tạo ra {df['session_id'].nunique()} sessions với khoảng nghỉ > {GAP_MINUTES} phút.")


print("   -> Chia dữ liệu thành tập train và validation...")
df['week'] = df['start_time_view'].dt.isocalendar().week
train_df = df[(df['week'] >= 11) & (df['week'] <= 24)]
val_df = df[(df['week'] >= 25) & (df['week'] <= 30)]
print(f"      - Tập Train (Tuần 11-24): {len(train_df)} bản ghi")
print(f"      - Tập Validation (Tuần 25-30): {len(val_df)} bản ghi")


train_path = 'data/interim/train.parquet'
val_path = 'data/interim/val.parquet'
train_df.to_parquet(train_path, index=False)
val_df.to_parquet(val_path, index=False)
print(f"      - Đã lưu tập train tại: {train_path}")
print(f"      - Đã lưu tập validation tại: {val_path}")

print("-" * 50)


   -> Đã tải thành công 2122836 bản ghi từ 'data/interim/logs_program.parquet'.
   -> Đang tạo session...
      - Đã tạo ra 1175897 sessions với khoảng nghỉ > 45 phút.
   -> Chia dữ liệu thành tập train và validation...
      - Tập Train (Tuần 11-24): 1647303 bản ghi
      - Tập Validation (Tuần 25-30): 475533 bản ghi
      - Đã lưu tập train tại: data/interim/train.parquet
      - Đã lưu tập validation tại: data/interim/val.parquet
--------------------------------------------------


**CF cơ bản/nâng cao**

In [6]:
# CÀI ĐẶT THƯ VIỆN VÀ TẠO THƯ MỤC
print("1. Cài đặt các thư viện cần thiết...")
# Cài đặt thư viện cho ALS, BPR, ItemKNN
!pip install implicit -q
# Cài đặt PyTorch cho LightGCN
!pip install torch -q

print("\n2. Tạo các thư mục đầu ra...")
import os
os.makedirs('data/candidates', exist_ok=True)
os.makedirs('candidates', exist_ok=True)
print("   -> Các thư mục đã sẵn sàng!")

1. Cài đặt các thư viện cần thiết...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

2. Tạo các thư mục đầu ra...
   -> Các thư mục đã sẵn sàng!



1.   TopPop



In [7]:
import pandas as pd
import numpy as np
import argparse
from pathlib import Path

def run(train_path: str, output_path: str, alpha_days: float):
    print(f"Bắt đầu chạy TopPop với alpha = {alpha_days} ngày...")
    train_df = pd.read_parquet(train_path)

    # Tính toán điểm phổ biến
    current_time = train_df['start_time_view'].max()

    train_df['delta_t'] = (current_time - train_df['start_time_view']).dt.total_seconds() / (24 * 3600)
    train_df['time_weight'] = np.exp(-train_df['delta_t'] / alpha_days)
    train_df['duration_weight'] = np.log1p(train_df['duration_view'])
    train_df['popularity_score'] = train_df['time_weight'] * train_df['duration_weight']
    item_popularity = train_df.groupby('tv_show_id')['popularity_score'].sum().sort_values(ascending=False).reset_index()

    # Lưu kết quả
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    item_popularity.to_parquet(output_path, index=False)
    print(f"Đã lưu bảng xếp hạng TopPop vào: {output_path}")
    print(f"Top 5 chương trình phổ biến nhất:\n{item_popularity.head()}")

train_path = "data/interim/train.parquet"
output_path = "data/candidates/toppop.parquet"
alpha = 15.0
run(train_path, output_path, alpha)

Bắt đầu chạy TopPop với alpha = 15.0 ngày...
Đã lưu bảng xếp hạng TopPop vào: data/candidates/toppop.parquet
Top 5 chương trình phổ biến nhất:
   tv_show_id  popularity_score
0   2400480.0      57308.211900
1    240081.0      33204.095094
2   6500479.0      27835.996343
3   2400508.0      22541.891228
4   2400467.0      20508.352953




2.   ItemKNN




In [8]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from implicit.nearest_neighbours import CosineRecommender
from pathlib import Path
from tqdm import tqdm

def build_user_item_matrix(df: pd.DataFrame):

    df.dropna(subset=['user_id', 'tv_show_id', 'duration_view'], inplace=True)

    df['user_id'] = df['user_id'].astype('category')
    df['tv_show_id'] = df['tv_show_id'].astype('category')

    user_codes = df['user_id'].cat.codes
    item_codes = df['tv_show_id'].cat.codes

    matrix = sp.csr_matrix(
        (np.log1p(df['duration_view']), (user_codes, item_codes)),
        shape=(len(df['user_id'].cat.categories), len(df['tv_show_id'].cat.categories))
    )

    user_map = {cat: i for i, cat in enumerate(df['user_id'].cat.categories)}
    rev_item_map = {i: cat for i, cat in enumerate(df['tv_show_id'].cat.categories)}

    return matrix, user_map, rev_item_map

def run(train_path: str, output_path: str, K: int, num_candidates: int):
    print(f"Bắt đầu chạy ItemKNN với K = {K}...")
    train_df = pd.read_parquet(train_path)

    user_item_matrix, user_map, rev_item_map = build_user_item_matrix(train_df)
    num_items = len(rev_item_map) # Tổng số item hợp lệ

    # Khởi tạo và huấn luyện mô hình
    model = CosineRecommender(K=K)
    model.fit(user_item_matrix.T.tocsr())
    print("Huấn luyện mô hình ItemKNN hoàn tất.")

    # Lặp qua từng người dùng để tạo gợi ý
    print("Tạo gợi ý cho từng người dùng...")
    candidates = []
    original_user_ids = list(user_map.keys())
    skipped_count = 0

    for original_user_id in tqdm(original_user_ids, desc="Generating recommendations"):
        user_idx = user_map[original_user_id]

        ids, scores = model.recommend(
            userid=user_idx,
            user_items=user_item_matrix[user_idx],
            N=num_candidates,
            filter_already_liked_items=True
        )

        for rank, item_idx in enumerate(ids):
            if item_idx in rev_item_map:
                candidates.append({
                    "user_id": original_user_id,
                    "tv_show_id": rev_item_map[item_idx],
                    "rank": rank + 1
                })
            else:
                skipped_count += 1

    candidate_df = pd.DataFrame(candidates)
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    candidate_df.to_parquet(output_path, index=False)
    print(f"Đã lưu {len(candidate_df)} gợi ý ItemKNN vào: {output_path}")
    if skipped_count > 0:
        print(f"CẢNH BÁO: Đã bỏ qua {skipped_count} gợi ý không hợp lệ do model trả về.")

# 1. Định nghĩa các tham số
train_path = "data/interim/train.parquet"
output_path = "data/candidates/itemknn.parquet"
k_neighbors = 200
num_cands = 50

# 2. Gọi hàm run
run(train_path, output_path, k_neighbors, num_cands)

/usr/local/lib/python3.12/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/tmp/pip-install-1ol76jmv/implicit_a74abdeac3184573b2b4894e1f4d359f/./implicit/gpu/utils.h:71)'
  warnings.warn(


Bắt đầu chạy ItemKNN với K = 200...


/usr/local/lib/python3.12/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.03190302848815918 seconds
  warnings.warn(


  0%|          | 0/4851 [00:00<?, ?it/s]

Huấn luyện mô hình ItemKNN hoàn tất.
Tạo gợi ý cho từng người dùng...


Generating recommendations: 100%|██████████| 4851/4851 [00:02<00:00, 1755.25it/s]


Đã lưu 185241 gợi ý ItemKNN vào: data/candidates/itemknn.parquet
CẢNH BÁO: Đã bỏ qua 57309 gợi ý không hợp lệ do model trả về.





3.   ALS/WMF



In [9]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from implicit.als import AlternatingLeastSquares
from pathlib import Path
from tqdm import tqdm

def build_user_item_matrix(df: pd.DataFrame):

    df_clean = df.dropna(subset=['user_id', 'tv_show_id', 'duration_view']).copy()

    unique_users = sorted(df_clean['user_id'].unique())
    unique_items = sorted(df_clean['tv_show_id'].unique())

    user_map = {user_id: i for i, user_id in enumerate(unique_users)}
    item_map = {item_id: i for i, item_id in enumerate(unique_items)}

    user_codes = df_clean['user_id'].map(user_map)
    item_codes = df_clean['tv_show_id'].map(item_map)

    matrix = sp.csr_matrix(
        (np.log1p(df_clean['duration_view']), (user_codes, item_codes)),
        shape=(len(unique_users), len(unique_items))
    )

    rev_item_map = {i: item_id for item_id, i in item_map.items()}

    return matrix, user_map, rev_item_map

def run(train_path: str, output_path: str, factors: int, reg: float, iters: int, alpha: float, num_candidates: int):
    print(f"Bắt đầu chạy ALS/WMF với factors = {factors}...")
    train_df = pd.read_parquet(train_path)

    user_item_matrix, user_map, rev_item_map = build_user_item_matrix(train_df)

    # Khởi tạo và huấn luyện mô hình
    model = AlternatingLeastSquares(
        factors=factors,
        regularization=reg,
        iterations=iters,
        random_state=42,
        use_gpu=False
    )
    model.fit(user_item_matrix.T.tocsr() * alpha)
    print("Huấn luyện mô hình ALS/WMF hoàn tất.")

    # Lấy số lượng user thực tế mà model đã được huấn luyện
    num_model_users = model.user_factors.shape[0]
    print(f"Model được huấn luyện với {num_model_users} users. Đặt giới hạn truy cập tại index {num_model_users - 1}.")

    # Tạo gợi ý cho từng người dùng
    print("Tạo gợi ý cho từng người dùng...")
    candidates = []
    original_user_ids = list(user_map.keys())
    skipped_users = 0

    for original_user_id in tqdm(original_user_ids, desc="Generating recommendations"):
        user_idx = user_map[original_user_id]

        if user_idx < num_model_users:
            ids, scores = model.recommend(
                userid=user_idx,
                user_items=user_item_matrix[user_idx],
                N=num_candidates,
                filter_already_liked_items=True
            )

            for rank, item_idx in enumerate(ids):
                if item_idx in rev_item_map:
                    candidates.append({
                        "user_id": original_user_id,
                        "tv_show_id": rev_item_map[item_idx],
                        "rank": rank + 1
                    })
        else:
            skipped_users += 1

    if skipped_users > 0:
        print(f"CẢNH BÁO: Đã bỏ qua {skipped_users} người dùng do có ID nằm ngoài giới hạn của mô hình.")

    candidate_df = pd.DataFrame(candidates)
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    candidate_df.to_parquet(output_path, index=False)
    print(f"Đã lưu {len(candidate_df)} gợi ý ALS/WMF vào: {output_path}")

# 1. Định nghĩa các tham số
train_path = "data/interim/train.parquet"
output_path = "data/candidates/als_wmf.parquet"
factors = 64
regularization = 0.05
iterations = 15
alpha = 1.0
num_cands = 50

# 2. Gọi hàm run
run(train_path, output_path, factors, regularization, iterations, alpha, num_cands)


Bắt đầu chạy ALS/WMF với factors = 64...


/usr/local/lib/python3.12/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

Huấn luyện mô hình ALS/WMF hoàn tất.
Model được huấn luyện với 3613 users. Đặt giới hạn truy cập tại index 3612.
Tạo gợi ý cho từng người dùng...


Generating recommendations: 100%|██████████| 4851/4851 [00:01<00:00, 4442.94it/s]


CẢNH BÁO: Đã bỏ qua 1238 người dùng do có ID nằm ngoài giới hạn của mô hình.
Đã lưu 135874 gợi ý ALS/WMF vào: data/candidates/als_wmf.parquet




4.   BPR-MF



In [10]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from implicit.bpr import BayesianPersonalizedRanking
from pathlib import Path
from tqdm import tqdm

def build_user_item_matrix(df: pd.DataFrame):
    """
    Xây dựng ma trận User-Item và các ánh xạ ID theo cách thủ công và an toàn nhất.
    """
    df_clean = df.dropna(subset=['user_id', 'tv_show_id', 'duration_view']).copy()

    unique_users = sorted(df_clean['user_id'].unique())
    unique_items = sorted(df_clean['tv_show_id'].unique())

    user_map = {user_id: i for i, user_id in enumerate(unique_users)}
    item_map = {item_id: i for i, item_id in enumerate(unique_items)}

    user_codes = df_clean['user_id'].map(user_map)
    item_codes = df_clean['tv_show_id'].map(item_map)

    matrix = sp.csr_matrix(
        (np.log1p(df_clean['duration_view']), (user_codes, item_codes)),
        shape=(len(unique_users), len(unique_items))
    )

    rev_item_map = {i: item_id for item_id, i in item_map.items()}

    return matrix, user_map, rev_item_map

def run(train_path: str, output_path: str, factors: int, lr: float, reg: float, iters: int, num_candidates: int):
    print(f"Bắt đầu chạy BPR-MF với factors = {factors}...")
    train_df = pd.read_parquet(train_path)

    user_item_matrix, user_map, rev_item_map = build_user_item_matrix(train_df)

    # Khởi tạo và huấn luyện mô hình
    model = BayesianPersonalizedRanking(
        factors=factors,
        learning_rate=lr,
        regularization=reg,
        iterations=iters,
        random_state=42,
        use_gpu=False
    )

    model.fit(user_item_matrix)
    print("Huấn luyện mô hình BPR-MF hoàn tất.")

    # Tạo gợi ý
    original_user_ids = list(user_map.keys())

    ids = model.recommend_all(
        user_items=user_item_matrix,
        N=num_candidates,
        filter_already_liked_items=True
    )

    # Chuyển đổi kết quả
    candidates = []
    for i, original_user_id in enumerate(original_user_ids):
        for rank, item_idx in enumerate(ids[i]):
            if item_idx in rev_item_map:
                candidates.append({
                    "user_id": original_user_id,
                    "tv_show_id": rev_item_map[item_idx],
                    "rank": rank + 1
                })

    candidate_df = pd.DataFrame(candidates)
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    candidate_df.to_parquet(output_path, index=False)
    print(f"Đã lưu {len(candidate_df)} gợi ý BPR-MF vào: {output_path}")

# Định nghĩa các tham số
train_path = "data/interim/train.parquet"
output_path = "data/candidates/bpr.parquet"
factors = 128
learning_rate = 1e-3
regularization = 1e-4
iterations = 100
num_cands = 50

run(train_path, output_path, factors, learning_rate, regularization, iterations, num_cands)


Bắt đầu chạy BPR-MF với factors = 128...


  0%|          | 0/100 [00:00<?, ?it/s]

Huấn luyện mô hình BPR-MF hoàn tất.
Đã lưu 242550 gợi ý BPR-MF vào: data/candidates/bpr.parquet




5.   LightGCN



In [11]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from collections import defaultdict
import argparse
from pathlib import Path
from tqdm import tqdm

class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, adj_matrix, embed_size=64, num_layers=3):
        super().__init__()
        self.num_users, self.num_items = num_users, num_items
        self.embed_size, self.num_layers = embed_size, num_layers
        self.adj_matrix = adj_matrix
        self.embedding = nn.Embedding(num_users + num_items, embed_size)
        nn.init.normal_(self.embedding.weight, std=0.1)

    def forward(self):
        all_embeddings = [self.embedding.weight]
        for _ in range(self.num_layers):
            new_embeddings = torch.sparse.mm(self.adj_matrix, all_embeddings[-1])
            all_embeddings.append(new_embeddings)
        final_embeddings = torch.stack(all_embeddings, dim=1).mean(dim=1)
        return torch.split(final_embeddings, [self.num_users, self.num_items])

def get_graph_and_dataloader(df, batch_size):
    user_map = {id: i for i, id in enumerate(df['user_id'].unique())}
    item_map = {id: i for i, id in enumerate(df['tv_show_id'].unique())}
    df['user_idx'], df['item_idx'] = df['user_id'].map(user_map), df['tv_show_id'].map(item_map)
    num_users, num_items = len(user_map), len(item_map)
    adj = sp.dok_matrix((num_users + num_items, num_users + num_items), dtype=np.float32)
    adj[:num_users, num_users:] = sp.csr_matrix((np.ones(len(df)), (df['user_idx'], df['item_idx'])), shape=(num_users, num_items))
    adj[num_users:, :num_users] = adj[:num_users, num_users:].T
    rowsum = np.array(adj.sum(axis=1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    norm_adj = d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt).tocoo()
    indices, values = torch.LongTensor(np.vstack((norm_adj.row, norm_adj.col))), torch.FloatTensor(norm_adj.data)
    adj_tensor = torch.sparse.FloatTensor(indices, values, torch.Size(norm_adj.shape))
    class BPRDataset(Dataset):
        def __init__(self, df, num_items):
            self.user_pos_items = df.groupby('user_idx')['item_idx'].apply(list).to_dict()
            self.users, self.num_items = list(self.user_pos_items.keys()), num_items
        def __len__(self): return len(self.users)
        def __getitem__(self, idx):
            user = self.users[idx]
            pos_item = np.random.choice(self.user_pos_items[user])
            neg_item = np.random.randint(0, self.num_items)
            while neg_item in self.user_pos_items[user]: neg_item = np.random.randint(0, self.num_items)
            return user, pos_item, neg_item
    return adj_tensor, DataLoader(BPRDataset(df, num_items), batch_size, True), user_map, item_map

def run(train_path, output_path, emb_size, layers, lr, reg, epochs, batch_size, num_candidates):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Bắt đầu chạy LightGCN trên thiết bị: {device}")
    df = pd.read_parquet(train_path)
    adj, dataloader, user_map, item_map = get_graph_and_dataloader(df, batch_size)
    num_users, num_items = len(user_map), len(item_map)
    rev_item_map = {i: id for id, i in item_map.items()}
    model = LightGCN(num_users, num_items, adj.to(device), emb_size, layers).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    print("Bắt đầu huấn luyện LightGCN...")
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for users, pos_items, neg_items in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            users, pos_items, neg_items = users.to(device), pos_items.to(device), neg_items.to(device)
            optimizer.zero_grad()
            user_embed, item_embed = model()
            u_e, pos_e, neg_e = user_embed[users], item_embed[pos_items], item_embed[neg_items]
            pos_scores, neg_scores = torch.mul(u_e, pos_e).sum(dim=1), torch.mul(u_e, neg_e).sum(dim=1)
            loss = -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean()
            l2_loss = (u_e.norm(2).pow(2) + pos_e.norm(2).pow(2) + neg_e.norm(2).pow(2)) / len(users)
            loss += reg * l2_loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")
    print("Tạo gợi ý cho tất cả người dùng...")
    model.eval()
    with torch.no_grad(): final_user_embed, final_item_embed = model()
    scores_matrix = torch.matmul(final_user_embed, final_item_embed.T)
    candidates = []
    original_user_ids = list(user_map.keys())
    user_pos_items = df.groupby('user_idx')['item_idx'].apply(list).to_dict()
    for user_idx, scores in enumerate(tqdm(scores_matrix, desc="Generating candidates")):
        if user_idx in user_pos_items: scores[user_pos_items[user_idx]] = -np.inf
        _, top_k_indices = torch.topk(scores, k=num_candidates)
        original_user_id = original_user_ids[user_idx]
        for rank, item_idx in enumerate(top_k_indices.cpu().numpy()):
            candidates.append({"user_id": original_user_id, "tv_show_id": rev_item_map[item_idx], "rank": rank + 1})
    candidate_df = pd.DataFrame(candidates)
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    candidate_df.to_parquet(output_path, index=False)
    print(f"Đã lưu {len(candidate_df)} gợi ý LightGCN vào: {output_path}")

# Định nghĩa các tham số
train_path = "data/interim/train.parquet"
output_path = "data/candidates/lightgcn.parquet"
emb_size = 64
layers = 3
learning_rate = 1e-3
regularization = 1e-4
epochs = 10
batch_size = 2048
num_cands = 50

run(train_path, output_path, emb_size, layers, learning_rate, regularization, epochs, batch_size, num_cands)

Bắt đầu chạy LightGCN trên thiết bị: cpu


/tmp/ipython-input-3248444499.py:44: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:653.)
  adj_tensor = torch.sparse.FloatTensor(indices, values, torch.Size(norm_adj.shape))


Bắt đầu huấn luyện LightGCN...


Epoch 1/10: 100%|██████████| 3/3 [00:01<00:00,  1.55it/s]


Epoch 1, Loss: 0.6927


...

Epoch 9, Loss: 0.6819


Epoch 10/10: 100%|██████████| 3/3 [00:01<00:00,  1.50it/s]


Epoch 10, Loss: 0.6783
Tạo gợi ý cho tất cả người dùng...


Generating candidates: 100%|██████████| 4851/4851 [00:01<00:00, 4391.76it/s]


Đã lưu 242550 gợi ý LightGCN vào: data/candidates/lightgcn.parquet
